<a href="https://colab.research.google.com/github/JAtharva22/SSD-AJ/blob/master/colab_nbs/grid_search_code_4mods_arush.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -L -o VOC2007.zip "https://www.dropbox.com/scl/fi/rpo43f2q77wpmdshv74o4/VOC2007.zip?rlkey=6amuy7ki8bnd7gg4q9lp59iik&st=v5nymauj&dl=1"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    17  100    17    0     0     27      0 --:--:-- --:--:-- --:--:--    27
100   496    0   496    0     0    320      0 --:--:--  0:00:01 --:--:--   547
100  624M  100  624M    0     0  49.8M      0  0:00:12  0:00:12 --:--:-- 43.0M


In [2]:
!unzip /content/VOC2007.zip

Streaming output truncated to the last 5000 lines.
  inflating: content/VOC2007/JPEGImages/knife_46_jpg.rf.bb1053d803c17d094846678f2471e7e9.jpg  
  inflating: content/VOC2007/JPEGImages/armas-74-_jpg.rf.f471a1f29f0d1c1e04f8e3030fe988bf.jpg  
  inflating: content/VOC2007/JPEGImages/Image_71_jpg.rf.cb5c7e85d1306b0f310a475969cd57c8.jpg  
  inflating: content/VOC2007/JPEGImages/knife_147_jpg.rf.d9a77fc7a3061c42945590efd20cf0ea.jpg  
  inflating: content/VOC2007/JPEGImages/0090__rot90__trans40_20_jpg.rf.ab11033209d9714536bae23227f0f778.jpg  
  inflating: content/VOC2007/JPEGImages/2-328-_jpg.rf.eaf4eca30aef55f3d62cce0cf14313a9.jpg  
  inflating: content/VOC2007/JPEGImages/0082__fliph__rot-45_jpg.rf.c574c41e4b892c88d90e472a32c06cb4.jpg  
  inflating: content/VOC2007/JPEGImages/knife-386-_jpg.rf.8540a315c3182f39a49f8885a7769c8a.jpg  
  inflating: content/VOC2007/JPEGImages/DSC_00511_JPG.rf.446cb948ee356afa25ee7745c82cab4d.jpg  
  inflating: content/VOC2007/JPEGImages/17437_jpg.rf.934d67745701

In [3]:
!mv /content/content/VOC2007 /content

In [4]:
import os
def count_lines_in_txt(txt_file_path):
    # Open the txt file and count the lines
    with open(txt_file_path, 'r') as file:
        lines = file.readlines()
        return len(lines)

# Example usage
txt_file_path = [
    "/content/VOC2007/ImageSets/Main/train.txt",
    "/content/VOC2007/ImageSets/Main/test.txt",
    "/content/VOC2007/ImageSets/Main/valid.txt"
]  # Ra with the path to your .txt file

for i in txt_file_path:
    line_count = count_lines_in_txt(i)
    print(f"The file contains {line_count} lines.")

The file contains 10814 lines.
The file contains 1352 lines.
The file contains 1352 lines.


In [5]:
import os

def get_folder_size(folder_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for file in filenames:
            file_path = os.path.join(dirpath, file)
            if os.path.exists(file_path):
                total_size += os.path.getsize(file_path)
    return total_size

# Example usage
folder_path = '/content/VOC2007'  # Replace with your folder path
folder_size = get_folder_size(folder_path)

# Convert the size to MB
folder_size_mb = folder_size / (1024 * 1024)
print(f"Folder size: {folder_size_mb:.2f} MB")


Folder size: 643.59 MB


In [6]:
!sudo rm -rf /content/VOC2007.zip /content/sample_data/ /content/content/

In [7]:
import os
import xml.etree.ElementTree as ET

def create_ground_truth_files(valid_txt_path, annotations_folder, output_folder):
    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Read the valid.txt file to get the image names
    with open(valid_txt_path, 'r') as f:
        image_names = [line.strip() for line in f.readlines()]

    # Process each image in valid.txt
    for image_name in image_names:
        # Find the corresponding XML file in the Annotations folder
        xml_file_path = os.path.join(annotations_folder, image_name + ".xml")

        # Check if the XML file exists
        if os.path.exists(xml_file_path):
            # Parse the XML file
            tree = ET.parse(xml_file_path)
            root = tree.getroot()

            # Get image width and height from the XML
            image_width = int(root.find("size").find("width").text)
            image_height = int(root.find("size").find("height").text)

            # Create the corresponding ground truth file
            ground_truth_file_path = os.path.join(output_folder, image_name + ".txt")

            with open(ground_truth_file_path, 'w') as gt_file:
                # Loop through all objects in the XML file
                for obj in root.findall('object'):
                    class_name = obj.find('name').text
                    bndbox = obj.find('bndbox')
                    xmin = int(bndbox.find('xmin').text)
                    ymin = int(bndbox.find('ymin').text)
                    xmax = int(bndbox.find('xmax').text)
                    ymax = int(bndbox.find('ymax').text)

                    # Normalize the bounding box coordinates
                    normalized_xmin = xmin / image_width
                    normalized_ymin = ymin / image_height
                    normalized_xmax = xmax / image_width
                    normalized_ymax = ymax / image_height

                    # Check for difficult tag
                    difficult = obj.find('difficult')
                    if difficult is not None and difficult.text == '1':
                        # If difficult, add the 'difficult' tag
                        gt_file.write(f"{class_name} {normalized_xmin} {normalized_ymin} {normalized_xmax} {normalized_ymax} difficult\n")
                    else:
                        # If not difficult, write normally
                        gt_file.write(f"{class_name} {normalized_xmin} {normalized_ymin} {normalized_xmax} {normalized_ymax}\n")

            print(f"Ground truth file created: {ground_truth_file_path}")


# Example usage:
valid_txt_path = '/content/VOC2007/ImageSets/Main/valid.txt'  # Path to your valid.txt file
annotations_folder = '/content/VOC2007/Annotations'  # Path to the annotations folder
output_folder = '/content/map_calc/gt'  # Path where ground truth files will be saved

create_ground_truth_files(valid_txt_path, annotations_folder, output_folder)

Ground truth file created: /content/map_calc/gt/291ac648a2f0762f_jpg.rf.d48f59d81bdcfe3c43aca8987b4fae3f.txt
Ground truth file created: /content/map_calc/gt/armas-2996-_jpg.rf.84390a7e7d32327c6443ff82a22e55e1.txt
Ground truth file created: /content/map_calc/gt/knife_233_jpg.rf.18dc9c3938a25f6a62424916abf0a2c7.txt
Ground truth file created: /content/map_calc/gt/armas-1618-_jpg.rf.f405a56d702861dc807f64fb7578627f.txt
Ground truth file created: /content/map_calc/gt/Image_100_JPG.rf.b5c8370cad1fb49f6d85118adbcf1332.txt
Ground truth file created: /content/map_calc/gt/6TO0B6LHLRC7_jpg.rf.63c4e419452d759bee162659b1adfa53.txt
Ground truth file created: /content/map_calc/gt/knife_243_jpg.rf.29ff7fc63d2dca6f2b92636ceea30321.txt
Ground truth file created: /content/map_calc/gt/open51_png_jpg.rf.02986bb7ef071b676cb5314287792ab2.txt
Ground truth file created: /content/map_calc/gt/KravMagaKnifeDefenseTechniques115_jpg.rf.da17def4876bfcb3c49889a8c7d5d110.txt
Ground truth file created: /content/map_cal

In [8]:
!git clone https://github.com/JAtharva22/SSD-AJ.git

Cloning into 'SSD-AJ'...
remote: Enumerating objects: 1146, done.
remote: Counting objects: 100% (380/380), done.
remote: Compressing objects: 100% (141/141), done.
remote: Total 1146 (delta 287), reused 271 (delta 239), pack-reused 766 (from 2)
Receiving objects: 100% (1146/1146), 2.50 MiB | 11.18 MiB/s, done.
Resolving deltas: 100% (671/671), done.


In [9]:
!pip install -r /content/SSD-AJ/requirements.txt

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 54.0 MB/s eta 0:00:00
  Created wheel for vizer: filename=Vizer-0.1.5-py3-none-any.whl size=3481 sha256=d4b429a4c94cd06cc2306ae9290eb234904205a6ffdf49982fa69e1af8d7388d
  Stored in dir

In [10]:
import os

# Specify the current directory name and the new name
old_name = '/content/SSD-AJ'
new_name = '/content/SSD'

# Rename the directory
os.rename(old_name, new_name)

print(f"Directory renamed from {old_name} to {new_name}")


Directory renamed from /content/SSD-AJ to /content/SSD


In [11]:
import sys
sys.path.append('/content/SSD/detect_code')
sys.path.append('/content/SSD/ssd')
sys.path.append('/content/SSD')

In [12]:
!mkdir /content/nms

In [13]:
# Download VGG model
!curl -L -o /content/nms/vgg.pth "https://www.dropbox.com/scl/fi/6e1ql5jadhfh6jro9s4bp/vgg_0.55nms.pth?rlkey=0gaawnd7vm2vxs6nehh4jr72g&st=rczsj7si&dl=1"

# Download EfficientNet model
!curl -L -o /content/nms/efficientnet.pth "https://www.dropbox.com/scl/fi/te8zo3ujkdfqofrudkgji/efficientNet_o.55nms.pth?rlkey=2lj87xcucugur9oye585wi567&st=p99pgr4s&dl=1"

# Download MobileNetV3 model
!curl -L -o /content/nms/mobilenet.pth "https://www.dropbox.com/scl/fi/tuky05d2s2arauvbo6u27/mobilenet_v3_0.55.pth?rlkey=f3e0ok0unjnzcg8sroowu8ugj&st=bfj3es14&dl=1"

# Download ResNet50 model
!curl -L -o /content/nms/resnet.pth "https://www.dropbox.com/scl/fi/126lu72nnbrbfz59nyhh0/resnet50_0.55nms.pth?rlkey=4jrl4hdkdle2ej2z23p2fnihj&st=y0qa94ch&dl=1"



  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    17  100    17    0     0     32      0 --:--:-- --:--:-- --:--:--    32
100   496    0   496    0     0    366      0 --:--:--  0:00:01 --:--:--     0
100  183M  100  183M    0     0  25.5M      0  0:00:07  0:00:07 --:--:-- 36.7M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    17  100    17    0     0     30      0 --:--:-- --:--:-- --:--:--    30
100   496    0   496    0     0    312      0 --:--:--  0:00:01 --:--:--     0
100 89.1M  100 89.1M    0     0  25.9M      0  0:00:03  0:00:03 --:--:-- 96.4M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    17  100    17    0     0     37      0 --:--:

In [14]:
config_files = [
    "/content/SSD/configs/efficient_net_b3_ssd300_voc0712.yaml",
    "/content/SSD/configs/mobilenet_v3_ssd320_voc0712.yaml",
    "/content/SSD/configs/resnet_50_ssd300_voc0712.yaml",
    "/content/SSD/configs/vgg_ssd300_voc0712.yaml",
]

checkpoint_files = [
    "/content/nms/efficientnet.pth",
    "/content/nms/mobilenet.pth",
    "/content/nms/resnet.pth",
    "/content/nms/vgg.pth",
]

gt_folder = "/content/map_calc/gt"

In [15]:
import os

def get_image_paths_from_txt(txt_file, images_directory):
    image_paths = []

    # Read the txt file to get the list of image filenames
    with open(txt_file, 'r') as f:
        # Loop through each line in the txt file
        for line in f:
            # Clean up any extra whitespace and newlines
            image_filename = line.strip()

            # If filename doesn't already have an extension, add .jpg
            if not image_filename.lower().endswith('.jpg'):
                image_filename += '.jpg'

            # Create the full path of the image file
            full_image_path = os.path.join(images_directory, image_filename)

            # Check if the file exists (optional but good practice)
            if os.path.isfile(full_image_path):
                image_paths.append(full_image_path)

    return image_paths

# Example usage:
txt_file = '/content/VOC2007/ImageSets/Main/valid.txt'  # Path to the .txt file
images_directory = '/content/VOC2007/JPEGImages'  # Path to the images directory
image_paths = get_image_paths_from_txt(txt_file, images_directory)

# print the image paths to verify
print(image_paths[:5])


['/content/VOC2007/JPEGImages/291ac648a2f0762f_jpg.rf.d48f59d81bdcfe3c43aca8987b4fae3f.jpg', '/content/VOC2007/JPEGImages/armas-2996-_jpg.rf.84390a7e7d32327c6443ff82a22e55e1.jpg', '/content/VOC2007/JPEGImages/knife_233_jpg.rf.18dc9c3938a25f6a62424916abf0a2c7.jpg', '/content/VOC2007/JPEGImages/armas-1618-_jpg.rf.f405a56d702861dc807f64fb7578627f.jpg', '/content/VOC2007/JPEGImages/Image_100_JPG.rf.b5c8370cad1fb49f6d85118adbcf1332.jpg']


In [16]:
from SSD.ssd.default import class_names_defined
# Function to write results to a txt file in the specified output directory
def save_detection_results(img_path, boxes, confidences, classes, output_dir):
    # Get the base name of the image file (without extension) and append '.txt'
    base_name = os.path.basename(img_path)
    file_name = os.path.splitext(base_name)[0] + '.txt'

    # Create the full output path
    output_path = os.path.join(output_dir, file_name)

    # Open the output file for writing
    with open(output_path, 'w') as f:
        for box, confidence, class_id in zip(boxes, confidences, classes):
            left, top, right, bottom = box
            cls_name = class_names_defined[class_id]
            # Write the detection results in the desired format
            f.write(f"{cls_name} {confidence} {left} {top} {right} {bottom}\n")

    print(f"Results saved to: {output_path}")

In [17]:
import os

def compare_txt_files(dir1, dir2):
    # Get list of .txt files in both directories
    dir1_txt_files = {f for f in os.listdir(dir1) if f.endswith('.txt')}
    dir2_txt_files = {f for f in os.listdir(dir2) if f.endswith('.txt')}

    # Compare if both directories have the same .txt files
    return dir1_txt_files == dir2_txt_files


In [18]:
import os

def count_matching_files(gt_folder, pred_folder):
    gt_files = set(os.listdir(gt_folder))
    pred_files = set(os.listdir(pred_folder))

    common_files = gt_files.intersection(pred_files)
    missing_in_pred = gt_files - pred_files
    missing_in_gt = pred_files - gt_files

    print(f"Common files: {len(common_files)}")
    print(f"Missing in pred: {len(missing_in_pred)}")
    print(f"Missing in ground_truths: {len(missing_in_gt)}")


In [19]:
import os
import numpy as np
from SSD.ssd.default import class_names_defined

def iou(box1, box2):
    # Compute Intersection over Union between two boxes: [left, top, right, bottom]
    x_left = max(box1[0], box2[0])
    y_top = max(box1[1], box2[1])
    x_right = min(box1[2], box2[2])
    y_bottom = min(box1[3], box2[3])
    if x_right < x_left or y_bottom < y_top:
        return 0.0
    intersection_area = (x_right - x_left + 1) * (y_bottom - y_top + 1)
    box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)
    return intersection_area / float(box1_area + box2_area - intersection_area)

def load_ground_truths(gt_folder):
    """
    Loads ground truth boxes from text files.
    Returns a dict: {class_id: {image_filename: [ { 'bbox': [...], 'detected': False }, ... ] } }
    """
    gt_data = {}
    a = 0
    for filename in os.listdir(gt_folder):
        filepath = os.path.join(gt_folder, filename)
        with open(filepath, 'r') as f:
            lines = f.read().strip().splitlines()
            if a == 0:
                print(lines)
            for line in lines:
                parts = line.strip().split()
                cls = parts[0]
                box = list(map(float, parts[1:5]))
                if cls not in gt_data:
                    gt_data[cls] = {}
                if filename not in gt_data[cls]:
                    gt_data[cls][filename] = []
                gt_data[cls][filename].append({
                    'bbox': box,
                    'detected': False,
                    'difficult': False
                })
                if a == 0:
                    print(gt_data)
        a +=1

    return gt_data

def load_predictions(pred_folder, class_names_defined):
    """
    Loads prediction boxes from text files.
    Returns a dict: {class_id: [ { 'image_id': filename, 'confidence': float, 'bbox': [...] }, ... ] }
    """
    a = 0
    pred_data = {}
    for filename in os.listdir(pred_folder):
        filepath = os.path.join(pred_folder, filename)
        with open(filepath, 'r') as f:
            lines = f.read().strip().splitlines()
            if a == 0:
                print(lines)
            for line in lines:
                parts = line.strip().split()
                cls = parts[0]
                conf = float(parts[1])
                box = list(map(float, parts[2:6]))
                if cls not in pred_data:
                    pred_data[cls] = []
                pred_data[cls].append({
                    'image_id': filename,
                    'confidence': conf,
                    'bbox': box
                })
                if a == 0:
                    print(pred_data)
        a +=1

    return pred_data

def voc_ap(rec, prec):
    """
    Compute VOC AP given precision and recall.
    This uses the 11-point interpolation as used in the original VOC challenge.
    """
    # Append sentinel values at beginning and end
    mrec = np.concatenate(([0.], rec, [1.]))
    mprec = np.concatenate(([0.], prec, [0.]))
    # Make precision monotonically decreasing
    for i in range(len(mprec) - 1, 0, -1):
        mprec[i-1] = np.maximum(mprec[i-1], mprec[i])
    # Identify points where recall changes
    i = np.where(mrec[1:] != mrec[:-1])[0]
    # Compute AP as the sum of (recall change) * precision
    ap = np.sum((mrec[i + 1] - mrec[i]) * mprec[i + 1])
    return ap

def compute_map(gt_folder, pred_folder, iou_threshold=0.5):
    """
    Computes the mean Average Precision (mAP) over all classes.
    """
    gt_data = load_ground_truths(gt_folder)
    pred_data = load_predictions(pred_folder, class_names_defined)
    aps = []
    for cls in gt_data.keys():
        if cls not in pred_data:
            print(f"No predictions for class {cls}.")
            continue
        # Sort predictions by descending confidence
        predictions = sorted(pred_data[cls], key=lambda x: x['confidence'], reverse=True)

        # Count total non-difficult ground truth boxes for this class
        npos = sum([1 for image_id in gt_data[cls]
                       for obj in gt_data[cls][image_id] if not obj['difficult']])
        # Initialize true positive and false positive arrays
        tp = np.zeros(len(predictions))
        fp = np.zeros(len(predictions))

        for i, pred in enumerate(predictions):
            image_id = pred['image_id']
            max_iou = -np.inf
            jmax = -1
            if image_id in gt_data[cls]:
                gt_objs = gt_data[cls][image_id]
                for j, obj in enumerate(gt_objs):
                    current_iou = iou(pred['bbox'], obj['bbox'])
                    if current_iou > max_iou:
                        max_iou = current_iou
                        jmax = j
            if max_iou >= iou_threshold:
                if not gt_data[cls][image_id][jmax]['difficult']:
                    if not gt_data[cls][image_id][jmax]['detected']:
                        tp[i] = 1
                        gt_data[cls][image_id][jmax]['detected'] = True
                    else:
                        fp[i] = 1  # duplicate detection
                else:
                    # Ignore difficult samples (or treat as not affecting score)
                    pass
            else:
                fp[i] = 1

        # Compute cumulative true positives and false positives
        fp = np.cumsum(fp)
        tp = np.cumsum(tp)
        rec = tp / float(npos)
        prec = tp / np.maximum(tp + fp, np.finfo(np.float64).eps)
        ap = voc_ap(rec, prec)
        aps.append(ap)
        print(f"AP for class {cls}: {ap:.4f}")

    mAP = np.mean(aps) if aps else 0.
    print(f"\nMean Average Precision (mAP): {mAP:.4f}")
    return mAP


In [20]:
ind_model_threshold=[0.3,0.3,0.3,0.3] # for every individual model

########################################
# Grid Search Hyperparameters
########################################

weights_list = [
    [1, 2, 3, 4],
    [2, 3, 4, 5],
    [2, 4, 6, 8]
]
conf_filter = 0.25

# Adjusting IoU Thresholds

In [21]:
# change accordingly

iou_values = [0.4, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8]
# iou_values = iou_values[:2]
iou_values = iou_values[2:4] # -> arush
# iou_values = iou_values[4:6] # -> manas
# iou_values = iou_values[6:] # -> aish
print(iou_values)

[0.55, 0.6]


In [22]:
import os
from SSD.detect_code.detector_class import WeaponDetector
a = []
# Path where we save predictions for each grid-search combination
base_pred_dir = '/content/map_calc/pred_grid_search'
os.makedirs(base_pred_dir, exist_ok=True)

best_map = -1.0
best_params = (None, None)  # (iou_threshold, weights)

for iou_thr in iou_values:
    for wts in weights_list:
        # 1) Initialize or reinitialize the detector with new weights
        #    If your WeaponDetector constructor needs “model_weights_nums”
        #    you’d pass wts there. Adjust as needed for your code.
        detector_nums = WeaponDetector(
            config_files,
            checkpoint_files,
            wts
        )

        # 2) Create an output directory unique to this (iou_thr, wts) combo
        combo_name = f"iou_{iou_thr}_wts_{str(wts).replace(' ', '')}"
        op_dir_pred_nums = os.path.join(base_pred_dir, combo_name)
        os.makedirs(op_dir_pred_nums, exist_ok=True)

        # 3) Run detection on the entire validation set
        for img_path in image_paths:
            boxes, confidences, classes = detector_nums.detect(
                img_path=img_path,
                conf_filter=conf_filter,
                iou_threshold=iou_thr,         # <--- using the iou_thr here
                ind_model_threshold=ind_model_threshold
            )
            save_detection_results(img_path, boxes, confidences, classes, output_dir=op_dir_pred_nums)

        # 4) Evaluate performance (e.g., mAP) for this combination
        #    ground_truth_dir should point to your actual ground-truth annotations.
        #    The 'op_dir_pred_nums' folder has the predictions we just saved.
        gt_folder = "/content/map_calc/gt"
        current_map = compute_map(gt_folder, op_dir_pred_nums)

        print(f"IOU={iou_thr}, Weights={wts}, mAP={current_map}")
        a.append([current_map, (iou_thr, wts)])

        # 5) Track the best combination so far
        if current_map > best_map:
            best_map = current_map
            best_params = (iou_thr, wts)

Loading model 1: efficientnet.pth


Downloading: "http://storage.googleapis.com/public-models/efficientnet-b3-c8376fa2.pth" to /root/.torch/models/efficientnet-b3-c8376fa2.pth
100%|██████████| 47.1M/47.1M [00:00<00:00, 128MB/s]


Loaded pretrained weights for efficientnet-b3
Loading model 2: mobilenet.pth


Downloading: "https://github.com/d-li14/mobilenetv3.pytorch/raw/master/pretrained/mobilenetv3-large-1cd25616.pth" to /root/.torch/models/mobilenetv3-large-1cd25616.pth
100%|██████████| 21.1M/21.1M [00:00<00:00, 103MB/s]
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model 3: resnet.pth


...

[([[0.0050935447216033936, 0.07946053187052408, 0.9858382161458333, 0.9434156290690104]], tensor([2]), tensor([0.6747]))]

Predicting with model 2
0.3
[([[0.00141105055809021, 0.03499025106430054, 0.9469635009765625, 0.916566276550293]], tensor([1]), tensor([0.7189]))]

Predicting with model 3
0.3
[([[0.00268399715423584, 0.10724506378173829, 0.9593532307942708, 0.9097568766276042]], tensor([1]), tensor([0.4992]))]

Predicting with model 4
0.3
[([[-0.00755372683207194, 0.09112784067789713, 1.0048445638020833, 0.9631336466471354]], tensor([1]), tensor([0.9693]))]

Detection Results:
Detection 1: gun (Confidence: 0.6812)
Bounding Box: [8.87807284e-04 8.28217417e-02 9.79870975e-01 9.41571832e-01]
[1.]
Results saved to: /content/map_calc/pred_grid_search/iou_0.55_wts_[1,2,3,4]/armas-2996-_jpg.rf.84390a7e7d32327c6443ff82a22e55e1.txt

Predicting with model 1
0.3


/usr/local/lib/python3.11/dist-packages/ensemble_boxes/ensemble_boxes_wbf.py:51: UserWarning: X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')


[([[0.015150785446166992, 0.03082579294840495, 0.9395622762044271, 0.9879832967122396]], tensor([3]), tensor([0.8537]))]

Predicting with model 2
0.3
[([[0.002064228057861328, -0.0012888610363006592, 0.9406608581542969, 0.9564356803894043]], tensor([3]), tensor([0.8696]))]

Predicting with model 3
0.3
[([[0.027684688568115234, -0.011497228940327963, 0.7479977416992187, 0.9256833902994792]], tensor([3]), tensor([0.8888]))]

Predicting with model 4
0.3
[([[0.02901619593302409, 0.032247244517008465, 0.8843783569335938, 1.0033292643229166]], tensor([3]), tensor([0.9093]))]

Detection Results:
Detection 1: knife (Confidence: 0.8897)
Bounding Box: [0.02201755 0.01614137 0.85980088 0.96805584]
[3.]
Results saved to: /content/map_calc/pred_grid_search/iou_0.55_wts_[1,2,3,4]/knife_233_jpg.rf.18dc9c3938a25f6a62424916abf0a2c7.txt

Predicting with model 1
0.3


/usr/local/lib/python3.11/dist-packages/ensemble_boxes/ensemble_boxes_wbf.py:54: UserWarning: Y1 < 0 in box. Set it to 0.
  warnings.warn('Y1 < 0 in box. Set it to 0.')
/usr/local/lib/python3.11/dist-packages/ensemble_boxes/ensemble_boxes_wbf.py:63: UserWarning: Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')


Streaming output truncated to the last 5000 lines.
[([[0.029645086924235026, 0.3121483357747396, 0.9922944132486979, 0.7216062418619792]], tensor([2]), tensor([0.6043]))]

Predicting with model 2
0.3
[([[0.026172101497650146, 0.35800721645355227, 0.9804044723510742, 0.7217502593994141]], tensor([2]), tensor([0.7597]))]

Predicting with model 3
0.3
[([[-0.056926485697428385, 0.33516751607259115, 1.0117151896158854, 0.7449933369954427]], tensor([2]), tensor([0.6875]))]

Predicting with model 4
0.3
[([[0.04174655596415202, 0.36498186747233075, 1.004986775716146, 0.7527437337239583], [-0.02807843526204427, 0.38411145528157553, 0.8370370992024739, 0.5843676249186198]], tensor([2, 2]), tensor([0.8766, 0.3094]))]

Detection Results:
Detection 1: heavy-weapon (Confidence: 0.7693)
Bounding Box: [0.02652647 0.35145977 0.99552411 0.74209768]
[2.]
Results saved to: /content/map_calc/pred_grid_search/iou_0.6_wts_[2,4,6,8]/0045__fliph__rot-45_jpg.rf.9762352ef36853b67a69686ada072f10.txt

Predicting w

In [23]:
# After the grid search, print out the best hyperparameters
print("\n===== Grid Search Complete =====")
print(f"Best mAP = {best_map:.4f}")
print(f"Best params => IOU Threshold: {best_params[0]}, Weights: {best_params[1]}")
print(a)


===== Grid Search Complete =====
Best mAP = 0.7491
Best params => IOU Threshold: 0.55, Weights: [1, 2, 3, 4]
[[np.float64(0.7490913158891499), (0.55, [1, 2, 3, 4])], [np.float64(0.7422602466663243), (0.55, [2, 3, 4, 5])], [np.float64(0.7490913158891499), (0.55, [2, 4, 6, 8])], [np.float64(0.7407390727794437), (0.6, [1, 2, 3, 4])], [np.float64(0.7337873785873238), (0.6, [2, 3, 4, 5])], [np.float64(0.7407390727794437), (0.6, [2, 4, 6, 8])]]
